In [ ]:
import os
import glob
import cv2
import numpy as np
import torch
from torchvision import transforms
from torchvision.models.segmentation import deeplabv3_resnet50
from ultralytics import YOLO
import mediapipe as mp
from skimage.feature import local_binary_pattern, graycomatrix, graycoprops
from skimage.color import rgb2gray
import matplotlib.pyplot as plt

# Paths - adjust to your local setup
INPUT_DIR = 'Time_12-34'              # folder containing JPEG frames
OUTPUT_DIR = 'output_segmentsa'      # folder where results will be saved
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Device for PyTorch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load YOLOv8n for person detection
yolo_model = YOLO('yolov8n.pt')

# Load DeepLabV3 (ResNet-50 backbone) for semantic segmentation
def load_segmentation_model():
    model = deeplabv3_resnet50(pretrained=True, progress=True)
    model.to(device)
    model.eval()
    return model

segmentation_model = load_segmentation_model()

# Preprocessing transform for DeepLabV3
seg_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((520, 520)),   # model input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Initialize MediaPipe Pose for body-part splitting
mp_pose = mp.solutions.pose.Pose(static_image_mode=True)
mp_drawing = mp.solutions.drawing_utils

# Helper: get segmentation mask via ResNet50 backbone
def get_person_mask_resnet(crop):
    img = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
    inp = seg_transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        out = segmentation_model(inp)['out'][0]  # [C, H, W]
        person_logits = out[15]
        person_prob = torch.sigmoid(person_logits)
        mask = person_prob.cpu().numpy() > 0.5
    mask = cv2.resize(mask.astype(np.uint8), (crop.shape[1], crop.shape[0]), interpolation=cv2.INTER_NEAREST)
    return mask.astype(bool)

# Helper: get pose landmarks
def get_landmarks(crop):
    rgb = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
    res = mp_pose.process(rgb)
    return res.pose_landmarks

# Helper: split mask into upper, lower, hands
def split_parts(mask, landmarks):
    H, W = mask.shape
    coords = {}
    for idx, lm in enumerate(landmarks.landmark):
        name = mp.solutions.pose.PoseLandmark(idx).name
        coords[name] = (int(lm.x * W), int(lm.y * H))

    hip_y = int((coords['LEFT_HIP'][1] + coords['RIGHT_HIP'][1]) / 2)
    fg = mask
    rows = np.arange(H)[:, None]
    upper = fg & (rows < hip_y)
    lower = fg & (rows >= hip_y)

    hands = np.zeros_like(fg, dtype=bool)
    r = int(0.1 * H)
    for side in ['LEFT', 'RIGHT']:
        key = f'{side}_WRIST'
        if key in coords:
            wx, wy = coords[key]
            y0, y1 = max(0, wy-r), min(H, wy+r)
            x0, x1 = max(0, wx-r), min(W, wx+r)
            hands[y0:y1, x0:x1] = True

    return upper, lower, hands, coords

# Plot HSV histogram
def plot_histogram(region_img, mask, title, save_path=None):
    hsv = cv2.cvtColor(region_img, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0], mask.astype(np.uint8), [180], [0, 180])
    plt.figure(figsize=(4, 2))
    plt.title(title)
    plt.xlabel("Hue")
    plt.ylabel("Frequency")
    plt.plot(hist, color='orange')
    plt.xlim([0, 180])
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path)
    plt.close()

# Feature extractor for color + texture
def extract_clothing_features(region_img, region_mask):
    region = cv2.bitwise_and(region_img, region_img, mask=region_mask.astype(np.uint8))
    hsv = cv2.cvtColor(region, cv2.COLOR_BGR2HSV)
    hist_h = cv2.calcHist([hsv], [0], region_mask.astype(np.uint8), [16], [0, 180])
    hist_s = cv2.calcHist([hsv], [1], region_mask.astype(np.uint8), [16], [0, 256])
    hist_v = cv2.calcHist([hsv], [2], region_mask.astype(np.uint8), [16], [0, 256])
    color_hist = np.concatenate([hist_h, hist_s, hist_v]).flatten()
    color_hist = color_hist / (np.sum(color_hist) + 1e-6)

    gray = rgb2gray(region)
    lbp = local_binary_pattern(gray, P=8, R=1, method='uniform')
    lbp_hist, _ = np.histogram(lbp[region_mask], bins=10, range=(0, 10), density=True)

    return np.concatenate([color_hist, lbp_hist])

# Texture features from grayscale image
def get_texture_features(gray_img):
    glcm = graycomatrix((gray_img * 255).astype(np.uint8), distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    return contrast, energy, homogeneity

# Process all JPEG frames
def process_all():
    imgs = glob.glob(os.path.join(INPUT_DIR, '**', '*.jpg'), recursive=True)
    for img_path in imgs:
        img = cv2.imread(img_path)
        if img is None:
            continue
        results = yolo_model(img)[0]
        person_boxes = [box.xyxy[0].cpu().numpy().astype(int)
                        for box, cls in zip(results.boxes, results.boxes.cls)
                        if int(cls) == 0]

        for i, (x1, y1, x2, y2) in enumerate(person_boxes):
            crop = img[y1:y2, x1:x2]
            mask = get_person_mask_resnet(crop)
            landmarks = get_landmarks(crop)
            if not landmarks:
                continue
            upper, lower, hands, coords = split_parts(mask, landmarks)

            # Feature extraction for clothing
            upper_feat = extract_clothing_features(crop, upper)
            lower_feat = extract_clothing_features(crop, lower)
            person_id_vector = np.concatenate([upper_feat, lower_feat])
            print(f"Person {i} features (Upper+Lower):", person_id_vector[:10])

            # Additional grayscale texture features
            gray_crop = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
            contrast, energy, homogeneity = get_texture_features(gray_crop)
            print(f"Person {i + 1} Texture - Contrast: {contrast}, Energy: {energy}, Homogeneity: {homogeneity}")

            # Draw lines for body parts
            vis = crop.copy()
            if 'LEFT_SHOULDER' in coords and 'RIGHT_SHOULDER' in coords:
                cv2.line(vis, coords['LEFT_SHOULDER'], coords['RIGHT_SHOULDER'], (0, 0, 255), 2)  # upper body line
            if 'LEFT_HIP' in coords and 'RIGHT_HIP' in coords:
                cv2.line(vis, coords['LEFT_HIP'], coords['RIGHT_HIP'], (0, 255, 0), 2)  # lower body line
            for side in ['LEFT', 'RIGHT']:
                key = f'{side}_WRIST'
                if key in coords:
                    cv2.circle(vis, coords[key], 5, (255, 0, 0), -1)  # hand points

            # Show and save histograms for upper, lower, and hands
            hist_dir = os.path.join(out_dir, "histograms")
            os.makedirs(hist_dir, exist_ok=True)
            plot_histogram(crop, upper, f"Person {i+1} Upper HSV Histogram", os.path.join(hist_dir, f"{basename}_p{i}_upper_hist.png"))
            plot_histogram(crop, lower, f"Person {i+1} Lower HSV Histogram", os.path.join(hist_dir, f"{basename}_p{i}_lower_hist.png"))
            plot_histogram(crop, hands, f"Person {i+1} Hand HSV Histogram", os.path.join(hist_dir, f"{basename}_p{i}_hand_hist.png"))

            rel_dir = os.path.relpath(os.path.dirname(img_path), INPUT_DIR)
            out_dir = os.path.join(OUTPUT_DIR, rel_dir)
            os.makedirs(out_dir, exist_ok=True)
            basename = os.path.splitext(os.path.basename(img_path))[0]
            out_path = os.path.join(out_dir, f"{basename}_p{i}.jpg")
            cv2.imwrite(out_path, vis)

if __name__ == '__main__':
    process_all()
    print("Processing complete. Segmented images saved to:", OUTPUT_DIR)



0: 480x640 2 persons, 3 cars, 38.2ms
Speed: 2.1ms preprocess, 38.2ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 persons, 1 car, 37.3ms
Speed: 1.8ms preprocess, 37.3ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)
Person 3 features (Upper+Lower): [   0.075738     0.21523    0.028669    0.011125   0.0017116   0.0008558           0           0           0           0]
Person 4 Texture - Contrast: 123.13685636856368, Energy: 0.03933042079593494, Homogeneity: 0.1721183819388773


UnboundLocalError: cannot access local variable 'out_dir' where it is not associated with a value